In [0]:
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, BatchNormalization,  MaxPool2D
from tensorflow.keras.datasets import mnist

from tensorflow.keras.utils import plot_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Reshaping to format which CNN expects (batch, height, width, channels)
xr_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
xr_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

# normalize inputs from 0-255 to 0-1
xr_train, xr_test = xr_train / 255.0, xr_test / 255.0

In [0]:
model = Sequential()

# Conv + Batch norm + Conv + MaxPool + Dropout
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'same', activation = 'relu', input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))  # Downscale
model.add(Dropout(rate = 0.25))
    
# Conv + Batch norm + Conv + MaxPool + Dropout
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))  # Downscale
model.add(Dropout(rate = 0.25))

# Flatten + Dense + D ropout + Dense (output)
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(rate = 0.5))
model.add(Dense(10, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
epochs = 50
datagen = ImageDataGenerator(
    rotation_range=10,  # Randomly rotate image within range
    zoom_range = 0.1,  # Randomly zoom in/out
    width_shift_range=0.1,  # Shift (fraction of total)
    height_shift_range=0.1,  # Shift (fraction of total)
)
datagen.fit(xr_train)

In [0]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.2, patience = 5, min_lr = 1e-5, verbose = 1)

In [0]:
history = model.fit_generator(datagen.flow(xr_train, y_train, batch_size = 64),
                              epochs=epochs, steps_per_epoch = xr_train.shape[0] // 64,
                              validation_data = (xr_test, y_test),
                              callbacks = [reduce_lr],
                              verbose = '1')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50


In [0]:
model.evaluate(xr_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0124 - accuracy: 0.9965


[0.012369989417493343, 0.9965000152587891]

In [0]:
model.save("mnist_CNN.h5")